# О Проекте: 

Запрос сформирован тем, что просмотр фильмов на оригинальном языке - это популярный и действенный метод прокачаться при изучении иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.

**Задача:**

- разработать ML решение для автоматического определения уровня сложности англоязычных фильмов по тексту субтитров. 

**Данные в распоряжении:**
- словари Oxford, в которых слова распределены по уровню сложности
- набор файлов-субтитров, рассортированных по каталогам в соответствии с уровнем сложности
- excel-файл со список несортированных фильмов и указанием их уровня без текста субтитров

**План выполнения задачи:**
- изучение и обработка предоставленного материала
- обработка текста субтитров и подготовка для машинного обучения
- тестирование модели и подбор гиперпараметров

**Обоснование выбора модели:**

***Алгоритм Doc2Vec***

В данной модели векторные представления документов обучаются предсказывать слова в документе, точнее берется вектор документа и объединяется с несколькими векторами слов из него, и модель пытается
предсказать следующее слово с учетом контекста. Данная особенность алгоритма позволит нам лучше определить отношение текста субтитров к тому или иному уровню английского языка. Для обучения мы воспользуемся исключительно текстовым содержанием фильмов.
Такие параметры как скорость речи или длительность реплики не берутся во внимание в данном случае. Это отдельная часть исследования, которое в нашем случае, опускается. Основной фокус внимания был сделан именно на текст и его анализ.

В качестве классификатора мы будем использовать RandomForestClassifier, который менее склонен к переобучению. В связи с малым количеством предоставленных данных воспользуемся более тонкими настройками модели. 

In [97]:
import os
import re
import pickle
import gensim
import warnings

import pandas as pd
import numpy as np

import multiprocessing
from tqdm import tqdm
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer


try:
    import spacy
except:
    !pip install spacy
    !python -m spacy download en
    !python -m spacy download en_core_web_sm
    import spacy

try:
    import chardet
except:
    !pip install chardet
    import chardet

try:
    from pypdf import PdfReader
except:
    !pip install pypdf
    from pypdf import PdfReader

try:
    import pysrt
except:
    !pip install pysrt
    import pysrt

In [98]:
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

nlp = spacy.load("en_core_web_sm")

SCORES_PATH    = 'English_level/English_scores'
SUBTITLES_PATH = 'English_level/English_scores/Subtitles_all'
OXFORD_PATH    = 'English_level/Oxford_CEFR_level'

ENGLISH_LEVELS = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

RANDOM_STATE = 1234

# Загрузка и обработка данных

## The Oxford CEFR level

В словарях Oxford 3000 и 5000 содержатся наиболее важные слова, которые должен знать каждый, кто учит английский.

В нашем случае, поскольку оценка фильма и присвоение ему уровня сложности будет происходить исключительно по тексту субтитров фильмов, часть предоставленных данных в исследовании учитываться не будут. 

## Файлы с субтитрами

Субтитры представлены в виде файлов .srt, где есть номер реплики, время и текст.

Файлы рассортированы по папкам, соответствующим уровням сложности языка.

Просканируем все файлы построчно:
- удалим из текста по возможности тэги, скобки, указания говорящего лица
- нетекстовые знаки, 
- лишние пробелы и переводы строки

In [ ]:
def process_line(line):
    if re.search(r'[A-Za-z]',line): 
        line = line.lower()
        line = re.sub(r'\n', ' ', line)                            # remove new lines
        line = re.sub(r'- ', ' ', line)                            # remove dash
        line = re.sub(r'\<[^\<]+?\>', '', line)                    # remove html tags
        line = re.sub(r'\([^\(]+?\)', '', line)                    # remove () parenthesis
        line = re.sub(r'\[[^\[]+?\]', '', line)                    # remove [] parenthesis
        line = re.sub(r'^([\w#\s]+\:)', ' ', line)                 # remove speaker tag
        line = re.sub(r'[^[:alnum:][:punct:][:blank:]]',' ', line) # remove all other non-speach shars
        line = re.sub(r'\s\s+', ' ', line).strip()                 # remove extra spaces
    return line


# обработка текста построчно
def process_text(content):
    text = []
    duration  = []
    for item in content:
        if not hasattr(item, 'duration'):
            print('no')
        if item.duration.ordinal>0:
            line = process_line(item.text_without_tags)
            text.append(line)
            duration.append(item.duration.ordinal/1000)
    return ' '.join(text), duration


# обработка файлов формата srt
def process_srt(dirname, filename):
    global count
    if not filename.endswith('.srt'):                        # skip non srt files
        print('Ошибка: файл', filename, 'другого формата')
        return False
    fullpath = os.path.join(dirname,filename)
    try:
        enc = chardet.detect(open(fullpath, "rb").read())['encoding']
        content = pysrt.open(fullpath, encoding=enc)
    except:
        print('Ошибка: файл не прочитан', filename)
        return False
    return process_text(content)                            # clean text and return


# movies dataset template
movies = pd.DataFrame(columns=['filename', 
                               'content', 
                               'duration', 
                               'level']
                     )

# recursive walk through dirs
for dirname, _, filenames in os.walk(SUBTITLES_PATH):
    for filename in filenames:
        level  = dirname.split('/')[-1]                   # get level name from dir
        result = process_srt(dirname, filename)           # process file
        if result:                                        # add movie to dataframe
            subs, duration = result
            movies.loc[len(movies)] = \
                {'filename' : filename.replace('.srt', ''),
                 'content'  : subs,
                 'duration' : duration, 
                 'level'    : level
                }

movies.head(3)

Ошибка: файл .DS_Store другого формата
Ошибка: файл .DS_Store другого формата


,filename,content,duration,level
0,"Crown, The S01E10 - Gloriana.en.FORCED",i am delighted to be here in cairo to meet wit...,"[3.04, 3.0, 2.96, 2.6, 2.24, 1.48, 1.24, 2.72,...",B2
1,"Crown, The S01E04 - Act of God.en",fuel on. fuel on. chocks are in position. swit...,"[2.12, 2.28, 1.84, 2.72, 5.6, 1.88, 2.8, 2.56,...",B2
2,Ghosts.of.Girlfriends.Past.2009.BluRay.720p.x2...,"good morning, connor. versace is on 1. okay. c...","[2.197, 1.433, 3.7, 2.299, 4.565, 3.335, 2.129...",B2


## Обработка excel-файла

В данном файле содержится список фильмов и уровни сложности.
 
Загрузим файл и изучим его:
- удалим дубликаты
- исправим несовпадения названий
- из указанных уровней оставим более сложный вариант
- удалим фильмы, для которых нет информации по уровню сложности

In [ ]:
movie_labels = pd.read_excel(f'{SCORES_PATH}/movies_labels.xlsx', index_col='id')
movie_labels.info()
movie_labels.columns = ['movie', 'level']
print('\nКоличество полных дубликатов:', movie_labels.duplicated().sum())
print('Количество дубликатов в названии фильмов:', movie_labels.movie.duplicated().sum())
movie_labels = movie_labels.drop_duplicates()
movie_labels.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 240
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   241 non-null    object
 1   Level   241 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB

Количество полных дубликатов: 2
Количество дубликатов в названии фильмов: 4


,movie,level
id,,
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


In [ ]:
# correct some mistakes in movies names
movie_labels.movie = movie_labels.movie.str.replace('.srt', '', regex=False)
movie_labels.loc[movie_labels.movie == 'Up (2009)', 'movie'] = 'Up(2009)'
movie_labels.loc[movie_labels.movie == 'The Grinch', 'movie'] = 'The.Grinch'

# удалим из level лишние символы, разделители оставим пробел
# из мультиуровней выберем наибольший
movie_labels.level = movie_labels.level \
                                 .str.replace(',', '', regex=False) \
                                 .str.replace('+', '', regex=False) \
                                 .str.replace('/', ' ', regex=False) \
                                 .str.split().transform(lambda x: max(x))

movie_labels.groupby('level').count().sort_index().style.format({'movie':'{:.0f} фильмов'})


,movie
level,
A2,32 фильмов
B1,58 фильмов
B2,109 фильмов
C1,40 фильмов


In [ ]:
# excel processing
for row in movie_labels.itertuples():

    n = movies.loc[movies.filename.str.contains(row.movie, regex=False)].shape[0]

    if n == 0:
        print('Текст не найден для фильма', row.movie)

    elif n == 1:
        selected_movie_level = movies.loc[
            movies.filename.str.contains(row.movie, regex=False), 'level'].values[0]

        if selected_movie_level == 'Subtitles':         # replace Subtitles with excel level
             movies.loc[
                 movies.filename.str.contains(row.movie, regex=False), 'level'] = row.level

        elif selected_movie_level != row.level:          # replace with max current level or excel
             movies.loc[
                 movies.filename.str.contains(row.movie, regex=False), 'level'
             ] = max(selected_movie_level, row.level)

    else:
        print('Текст есть в датасете', row.movie)


movies = movies[movies.level!='Subtitles']

Текст не найден для фильма The Secret Life of Pets.en
Текст не найден для фильма Glass Onion
Текст не найден для фильма Matilda(2022)
Текст не найден для фильма Bullet train
Текст не найден для фильма Thor: love and thunder
Текст не найден для фильма Lightyear


In [ ]:
movies['target'] = movies.level.map({'A1':0, 'A2':1, 'B1':2, 'B2':3, 'C1':4, 'C2':5})
movies = movies.fillna(0)

In [ ]:
movies['level'].value_counts()

B2    144
B1     55
C1     39
A2     32
Name: level, dtype: int64

In [ ]:
movies

,filename,content,duration,level,target
0,"Crown, The S01E10 - Gloriana.en.FORCED",i am delighted to be here in cairo to meet wit...,"[3.04, 3.0, 2.96, 2.6, 2.24, 1.48, 1.24, 2.72,...",B2,3
1,"Crown, The S01E04 - Act of God.en",fuel on. fuel on. chocks are in position. swit...,"[2.12, 2.28, 1.84, 2.72, 5.6, 1.88, 2.8, 2.56,...",B2,3
2,Ghosts.of.Girlfriends.Past.2009.BluRay.720p.x2...,"good morning, connor. versace is on 1. okay. c...","[2.197, 1.433, 3.7, 2.299, 4.565, 3.335, 2.129...",B2,3
3,Suits.S01E06.1080p.BluRay.AAC5.1.x265-DTG.02.EN,"harvey, i don't need a perp-walk or a front-pa...","[2.252, 1.208, 2.919, 3.044, 2.168, 2.668, 1.8...",B2,3
4,Suits.S02E04.HDTV.x264-ASAP,"i want to, uh... taupe. is that...? justice th...","[2.001, 3.036, 1.634, 0.999, 1.032, 2.167, 1.7...",B2,3
...,...,...,...,...,...
273,Suits S04E09 EngSub,previously on suits... i want you to decide if...,"[1.187, 4.012, 4.838, 1.878, 1.801, 3.386, 2.4...",C1,4
274,Suits.S03E09.480p.HDTV.x264-mSD,previously on suits... i'm bonding with your f...,"[1.454, 1.464, 1.692, 1.13, 1.3, 1.851, 2.786,...",C1,4
275,Suits S04E10 EngSub,"previously on suits... sheila amanda sazs, wil...","[1.205, 1.495, 1.177, 2.345, 2.478, 1.111, 2.7...",C1,4
276,Downton Abbey - S01E03 - Episode 3.eng.SDH,"there you are, mr. bates. it's in. came this m...","[2.708, 3.118, 1.832, 1.206, 2.287, 2.366, 1.7...",C1,4


Очевидным является дисбаланс классов, некоторые и вовсе не представлены - 'A1' и 'C2'. Это второй момент, который стоит иметь в виду, помимо очень скромного по объемам датасета для обучения и тестирования модели.

# Обучение модели

Настройка моделей обучения и оценки Doc2Vec


Векторное представление слов (англ. word embedding) — общее название для различных подходов к моделированию языка и обучению представлений в обработке естественного языка, направленных на сопоставление словам из некоторого словаря векторов небольшой размерности. 

Данный алгоритм сначала создает словарь, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, имеющие схожий смысл), в векторном представлении имеют высокое косинусное сходство. 
В отличие от мешка слов, матрица эмбедингов не разреженная +  учитывается контекст слова. Во время обучения мы будем использовать не слишком много эпох - чтобы в векторном пространстве слова далеко не разошлись - из-за этого может быть риск "плохого" представления слов для нас. Если бы выборка была значимо больше, то мы увеличили бы количетво эпох тренировки. 
Поэтому нам не нужно специально сопоставлять слова со словарем Oxford. К примеру, есть слово age как существительное, которое относится к уровню знаний A1. Но то же слово как глагол - это уже другой уровень, выше. 

Соответственно, данный факт необходимо учитывать при обучении. С данной особенностью языка отлично справляется модель. Стоит отметить, что классы дисбалансированы + некоторые имеют расхождение в трактовках "классов" - когда A2/A2+, B1. Вспомним "золотое правило машинного обучения" -  garbage in - garbage out. Т.е. для потенциального улучшения качества модели в будущем нам потребуется большее количество данных с качественной разметкой (экспертизой).

In [ ]:
##remove stop words from word list
def remove_stop_words(sample_words):
    stop_words = set(stopwords.words('english'))
    sample_words = [x for x in sample_words if not x in stop_words]
    return sample_words

##remove special characters from word list
def remove_special_char(sample_words):
    special_char = set(punctuation) 
    sample_words = [x for x in sample_words if not x in special_char]    
    return sample_words

##lemmatize words in word list
def lemmatizer(sample_words):
    lemmatizer = WordNetLemmatizer()
    sample_words = [lemmatizer.lemmatize(x) for x in sample_words]
    return sample_words

##all words in lower case
def lower_case(sample_words):
    sample_words = [x.lower() for x in sample_words]
    return sample_words

##normalize a word list (if document already tokenized)
def normalize_word_list(sample_words,
                        lowercase=True,
                        stopwords=True,
                        specialchar=True,
                        lemmatize=True):
    if lowercase:
        sample_words = lower_case(sample_words)
    if stopwords:
        sample_words = remove_stop_words(sample_words)
    if specialchar:
        sample_words = remove_special_char(sample_words)
    if lemmatize:
        sample_words = lemmatizer(sample_words)
    sample_words = ' '.join(sample_words)
    return sample_words

##normalize a list of sentences
def normalize_sent_list(sample_sents,
                        lowercase=True,
                        stopwords=True,
                        specialchar=True,
                        lemmatize=True):    
    print("Pre-processing text ...")
    sent_list = sample_sents
    for i in range(len(sample_sents)):
        sent_list[i] = re.findall(r"[\w']+|[.,!?;]", sent_list[i])
        sent_list[i] = normalize_word_list(sent_list[i],
                            lowercase=True,
                            stopwords=True,
                            specialchar=True,
                            lemmatize=True)
    return sent_list

In [ ]:
def prepare_training_data(data_file):
    ## read training data as a pandas dataframe
    orig_df = data_file
    ## text preprocessing
    text = pd.Series.tolist((orig_df['content']))
    x_target = pd.Series.tolist((orig_df['target']))
    x_level = pd.Series.tolist((orig_df['level']))
    
    text = normalize_sent_list(text,
                        		lowercase=True,
                        		stopwords=True,
                        		specialchar=True,
                        		lemmatize=False)
    ## preparing preprocessed text
    text_df = pd.DataFrame(text, columns=["content"])
    text_df['level'] = x_level
    text_df['target'] = x_target
    print(text_df)
    return text_df

def Doc2VecModel(text_df, no_epochs, val_split_ratio):
    ## splitting dataframe into training and validation frames
    train_df, val_df = train_test_split(text_df, test_size=val_split_ratio, random_state=RANDOM_STATE)	
    ## creating tagged documents
    train_tagged = train_df.apply(
        lambda r: TaggedDocument(words=r['content'].split(), tags=str(r.target)), axis=1)
    val_tagged = val_df.apply(
        lambda r: TaggedDocument(words=r['content'].split(), tags=str(r.target)), axis=1)
    ## building a distributed bag of words model 
    cores = multiprocessing.cpu_count()
    print("Building the Doc2Vec model vocab...")
    model_dbow = Doc2Vec(dm=0, vector_size=2000, negative=5, min_count=3, workers=cores)
    model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])
    ## training the model
    print("Training the Doc2Vec model for", no_epochs, "number of epochs" )
    for epoch in range(no_epochs):
        model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), 
                total_examples=len(train_tagged.values), epochs=1)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
    ## preparing document vectors for learning
    def vec_for_learning(model, tagged_docs):
        sents = tagged_docs.values
        targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
        return targets, regressors
    y_train, X_train = vec_for_learning(model_dbow, train_tagged)
    y_val, X_val = vec_for_learning(model_dbow, val_tagged)
    ## training RandomForestClassifier model
    print("Training RandomForestClassifier model...")
    rfc=RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
    param_grid = {
    'max_depth': [10],
    'max_features': [2, 3],
    'min_samples_leaf': [3],
    'min_samples_split': [5, 8, 10],
    'n_estimators': [100, 150],
    'criterion': ['entropy']
    }
    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, scoring = 'f1', cv=5)
    CV_rfc.fit(X_train, y_train)
    print(CV_rfc.best_params_)
    b_rfc = CV_rfc.best_estimator_
    ## making predictions on the training set
    print("Prediction numbers:")
    train_binary = b_rfc.predict(X_train)
    print('Accuracy on the training set : %s' % accuracy_score(y_train, train_binary))
    print('F1 score on the training set : {}'.format(f1_score(y_train, train_binary, average='weighted')))
    ## making predictions on the validation set
    val_binary = b_rfc.predict(X_val)
    print('Accuracy on the validation set : %s' % accuracy_score(y_val, val_binary))
    print('F1 score on the validation set : {}'.format(f1_score(y_val, val_binary, average='weighted')))
    return model_dbow, b_rfc

In [108]:
no_epochs = 3

val_split_ratio = 0.15

## preparing training data
text_df = prepare_training_data(movies)

## building the document vector model
model_dbow, classifier = Doc2VecModel(text_df, no_epochs,  val_split_ratio)

Pre-processing text ...
                                               content level  target
0    delighted cairo meet colonel nasser continue d...    B2       3
1    fuel fuel chock position switch sure sir got m...    B2       3
2    good morning connor versace 1 okay clear good ...    B2       3
3    harvey need perp walk front page headline want...    B2       3
4    want uh taupe justice thomas knew louis ah poi...    B2       3
..                                                 ...   ...     ...
265  previously suit want decide love hate want com...    C1       4
266  previously suit i'm bonding father speaking ta...    C1       4
267  previously suit sheila amanda sazs marry yes i...    C1       4
268  mr bates came morning said would quite thing h...    C1       4
269  got name wall want respect come meaning want o...    C1       4

[270 rows x 3 columns]
Building the Doc2Vec model vocab...


100%|█████████████████████████████████████| 229/229 [00:00<00:00, 553280.88it/s]


Training the Doc2Vec model for 3 number of epochs


100%|█████████████████████████████████████| 229/229 [00:00<00:00, 888525.08it/s]


Training RandomForestClassifier model...
{'criterion': 'entropy', 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 100}
Prediction numbers:
Accuracy on the training set : 1.0
F1 score on the training set : 1.0
Accuracy on the validation set : 0.6585365853658537
F1 score on the validation set : 0.5982632967284168


# Заключение

Как итог, мы получили в рамках ограниченных ресурсов, данных и неполноценной разметки, максимально возможный вариант эффективности модели. 
Так, мы получили на валидационной выборке результат F1 score приблизительно 60%, что можно считать более чем удовлетворительным при многоклассовой и несбалансированной выборке со спорной разметкой данных.